In [1]:
fa

In [2]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [10]:
df.drop('id',axis=1,inplace=True)

In [11]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['encoded_diagnosis'] = label_encoder.fit_transform(df['diagnosis'])

In [13]:
df.drop('diagnosis',axis=1,inplace=True)
df.drop('Unnamed: 32',axis=1,inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   radius_mean              569 non-null    float64
 1   texture_mean             569 non-null    float64
 2   perimeter_mean           569 non-null    float64
 3   area_mean                569 non-null    float64
 4   smoothness_mean          569 non-null    float64
 5   compactness_mean         569 non-null    float64
 6   concavity_mean           569 non-null    float64
 7   concave points_mean      569 non-null    float64
 8   symmetry_mean            569 non-null    float64
 9   fractal_dimension_mean   569 non-null    float64
 10  radius_se                569 non-null    float64
 11  texture_se               569 non-null    float64
 12  perimeter_se             569 non-null    float64
 13  area_se                  569 non-null    float64
 14  smoothness_se            5

In [15]:
X = df.drop('encoded_diagnosis', axis=1)
y = df['encoded_diagnosis']

In [16]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA
pca = PCA(n_components=0.95)  # Keep 95% of the variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Determine which features are not important
important_features = pca.components_

# Train SVM with linear kernel
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predict the labels for test set
y_pred = svm_model.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of SVM with PCA:", accuracy)

# Print the features that are not important
important_feature_indices = pca.explained_variance_ratio_.argsort()[:-1:-1]
print("Features that are not important:")
print(X.columns[important_feature_indices])

Accuracy of SVM with PCA: 0.9473684210526315
Features that are not important:
Index([], dtype='object')


In [19]:
# Train SVM with rbf kernel
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train_pca, y_train)

# Predict the labels for test set
y_pred = svm_rbf.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of SVM with PCA:", accuracy)

# Print the features that are not important
important_feature_indices = pca.explained_variance_ratio_.argsort()[:-1:-1]
print("Features that are not important:")
print(X.columns[important_feature_indices])

Accuracy of SVM with PCA: 0.9473684210526315
Features that are not important:
Index([], dtype='object')


In [20]:
from skrebate import ReliefF
# Apply ReliefF for feature selection
relief = ReliefF(n_features_to_select=10, n_neighbors=100)
X_train_relief = relief.fit_transform(X_train.values, y_train.values)

In [21]:
# Print features ranked by ReliefF
selected_features = X.columns[relief.top_features_]
unimportant_features = X.columns[~relief.top_features_]
print("Selected features:", selected_features)
print("Unimportant features:", unimportant_features)

Selected features: Index(['concave points_worst', 'concave points_mean', 'radius_worst',
       'perimeter_worst', 'perimeter_mean', 'radius_mean', 'concavity_mean',
       'area_mean', 'area_worst', 'concavity_worst', 'compactness_mean',
       'compactness_worst', 'texture_worst', 'smoothness_worst',
       'fractal_dimension_worst', 'radius_se', 'texture_mean',
       'symmetry_worst', 'area_se', 'perimeter_se', 'smoothness_mean',
       'concave points_se', 'fractal_dimension_mean', 'compactness_se',
       'symmetry_mean', 'symmetry_se', 'concavity_se', 'fractal_dimension_se',
       'smoothness_se', 'texture_se'],
      dtype='object')
Unimportant features: Index(['perimeter_mean', 'perimeter_worst', 'fractal_dimension_mean',
       'concave points_mean', 'concave points_worst',
       'fractal_dimension_worst', 'area_worst', 'concavity_worst',
       'concavity_mean', 'area_mean', 'smoothness_worst', 'smoothness_mean',
       'symmetry_mean', 'compactness_mean', 'radius_mean',
 

In [25]:
# Selecting features using ReliefF
selected_features = X.columns[relief.top_features_]

# Splitting data based on selected features
X_selected = X[selected_features]
y = df['encoded_diagnosis']  # Replace 'target_column' with the name of your target column

# Printing selected and unimportant features
unimportant_features = X.columns[~relief.top_features_]
print("Selected features:", selected_features)
print("Unimportant features:", unimportant_features)

Selected features: Index(['concave points_worst', 'concave points_mean', 'radius_worst',
       'perimeter_worst', 'perimeter_mean', 'radius_mean', 'concavity_mean',
       'area_mean', 'area_worst', 'concavity_worst', 'compactness_mean',
       'compactness_worst', 'texture_worst', 'smoothness_worst',
       'fractal_dimension_worst', 'radius_se', 'texture_mean',
       'symmetry_worst', 'area_se', 'perimeter_se', 'smoothness_mean',
       'concave points_se', 'fractal_dimension_mean', 'compactness_se',
       'symmetry_mean', 'symmetry_se', 'concavity_se', 'fractal_dimension_se',
       'smoothness_se', 'texture_se'],
      dtype='object')
Unimportant features: Index(['perimeter_mean', 'perimeter_worst', 'fractal_dimension_mean',
       'concave points_mean', 'concave points_worst',
       'fractal_dimension_worst', 'area_worst', 'concavity_worst',
       'concavity_mean', 'area_mean', 'smoothness_worst', 'smoothness_mean',
       'symmetry_mean', 'compactness_mean', 'radius_mean',
 

In [27]:
cleaned_df = pd.concat([X_selected, y], axis=1)

In [28]:
# Save cleaned_df as CSV
cleaned_df.to_csv('cleaned_data.csv', index=False) 

In [26]:
X_selected.head()

,concave points_worst,concave points_mean,radius_worst,perimeter_worst,perimeter_mean,radius_mean,concavity_mean,area_mean,area_worst,concavity_worst,...,smoothness_mean,concave points_se,fractal_dimension_mean,compactness_se,symmetry_mean,symmetry_se,concavity_se,fractal_dimension_se,smoothness_se,texture_se
0,0.2654,0.14710,25.38,184.60,122.80,17.99,0.3001,1001.0,2019.0,0.7119,...,0.11840,0.01587,0.07871,0.04904,0.2419,0.03003,0.05373,0.006193,0.006399,0.9053
1,0.1860,0.07017,24.99,158.80,132.90,20.57,0.0869,1326.0,1956.0,0.2416,...,0.08474,0.01340,0.05667,0.01308,0.1812,0.01389,0.01860,0.003532,0.005225,0.7339
2,0.2430,0.12790,23.57,152.50,130.00,19.69,0.1974,1203.0,1709.0,0.4504,...,0.10960,0.02058,0.05999,0.04006,0.2069,0.02250,0.03832,0.004571,0.006150,0.7869
3,0.2575,0.10520,14.91,98.87,77.58,11.42,0.2414,386.1,567.7,0.6869,...,0.14250,0.01867,0.09744,0.07458,0.2597,0.05963,0.05661,0.009208,0.009110,1.1560
4,0.1625,0.10430,22.54,152.20,135.10,20.29,0.1980,1297.0,1575.0,0.4000,...,0.10030,0.01885,0.05883,0.02461,0.1809,0.01756,0.05688,0.005115,0.011490,0.7813


In [22]:
# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_train_pca = pca.fit_transform(X_train_relief)
X_test_pca = pca.transform(relief.transform(X_test.values))

# Train SVM with RBF kernel
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train_pca, y_train)

# Make predictions
y_pred = svm_rbf.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of SVM with ReliefF, PCA, and RBF kernel:", accuracy)

Accuracy of SVM with ReliefF, PCA, and RBF kernel: 0.9473684210526315


In [23]:
# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_train_pca = pca.fit_transform(X_train_relief)
X_test_pca = pca.transform(relief.transform(X_test.values))

# Train SVM with RBF kernel
svm_linear = SVC(kernel='linear', random_state=42)
svm_linear.fit(X_train_pca, y_train)

# Make predictions
y_pred = svm_linear.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of SVM with ReliefF, PCA, and RBF Linear:", accuracy)

Accuracy of SVM with ReliefF, PCA, and RBF Linear: 0.9473684210526315
